In [ ]:
#!/usr/bin/env python3
import os
import logging
from pathlib import Path
import requests from optkeras.optkeras import OptKeras
import optkeras
import pickle
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from keras import layers
from keras.layers import Input,Dense,BatchNormalization,Flatten,Dropout,GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model,load_model
from keras.optimizers import RMSprop

import optuna
from zipfile import ZipFile

logging.basicConfig(level=logging.DEBUG)

#Import Pegasus API
from Pegasus.api import *

#Downloading dataset
zipurl = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip'    
r = requests.get(zipurl, stream = True) 
  
with open("dataset.zip","wb") as pdf: 
    for chunk in r.iter_content(chunk_size=1024): 
         if chunk: 
            pdf.write(chunk)
            
# Extract all the contents of zip file in current directory            
with ZipFile('dataset.zip', 'r') as zipObj:
    zipObj.extractall()

In [13]:
from optkeras.optkeras import OptKeras
import optkeras
import pickle
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from keras import layers
from keras.layers import Input,Dense,BatchNormalization,Flatten,Dropout,GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model,load_model
from keras.optimizers import RMSprop

import optuna

In [1]:
!sudo pip3 install --upgrade pip

Requirement already up-to-date: pip in /usr/local/lib/python3.6/site-packages (20.2.2)


In [2]:
!sudo pip3 install  keras==2.1.5 tensorflow numpy pandas pillow opencv-python sklearn 

     |████████████████████████████████| 334 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 320.4 MB 99 kB/s s eta 0:00:01    |██████▎                         | 63.2 MB 26.3 MB/s eta 0:00:10     |███████████▌                    | 114.9 MB 27.7 MB/s eta 0:00:08     |█████████████▊                  | 137.6 MB 24.0 MB/s eta 0:00:08     |████████████████████████████▏   | 282.3 MB 35.3 MB/s eta 0:00:02     |███████████████████████████████▌| 315.8 MB 21.5 MB/s eta 0:00:01
     |████████████████████████████████| 14.5 MB 32.5 MB/s eta 0:00:01
     |████████████████████████████████| 10.5 MB 38.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 37.5 MB/s eta 0:00:01
     |████████████████████████████████| 49.4 MB 32.3 MB/s eta 0:00:01     |████████████████████            | 30.8 MB 32.3 MB/s eta 0:00:01
     |████████████████████████████████| 25.9 MB 44.5 MB/s eta 0:00:01
     |████████████████████████████████| 127 kB 4.5 MB/s eta 0:00:01
     |█████████████████████

    Running setup.py install for termcolor ... done
    Running setup.py install for wrapt ... done
  Attempting uninstall: setuptools
    Found existing installation: setuptools 39.2.0
    Uninstalling setuptools-39.2.0:
      Successfully uninstalled setuptools-39.2.0
    Running setup.py install for sklearn ... done
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow 2.3.0 requires numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.1 which is incompatible.
tensorflow 2.3.0 requires scipy==1.4.1, but you'll have scipy 1.5.2 which is incompatible.


In [3]:
!sudo pip3 install keras==2.1.5 tensorflow==1.13.1 numpy pandas pillow sklearn optuna scikit-image ray ray[tune] 

     |████████████████████████████████| 92.5 MB 65.9 MB/s eta 0:00:01    |█████                           | 14.5 MB 1.8 MB/s eta 0:00:43     |████████▊                       | 25.1 MB 53.1 MB/s eta 0:00:02     |██████████▉                     | 31.2 MB 53.1 MB/s eta 0:00:02
     |████████████████████████████████| 226 kB 9.7 MB/s eta 0:00:01
     |████████████████████████████████| 12.4 MB 21.7 MB/s eta 0:00:01
     |████████████████████████████████| 22.0 MB 29.0 MB/s eta 0:00:01    |███▊                            | 2.6 MB 29.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 26.6 MB/s eta 0:00:01     |█████████████▊                  | 1.4 MB 26.6 MB/s eta 0:00:01
     |████████████████████████████████| 367 kB 28.8 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 12.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 28.9 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel

     |████████████████████████████████| 75 kB 5.6 MB/s eta 0:00:011
     |████████████████████████████████| 130 kB 50.0 MB/s eta 0:00:01
     |████████████████████████████████| 110 kB 42.6 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 2.9 MB/s  eta 0:00:01
     |████████████████████████████████| 88 kB 13.2 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 25.8 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 14.5 MB/s eta 0:00:01
     |████████████████████████████████| 460 kB 39.7 MB/s eta 0:00:01
     |████████████████████████████████| 257 kB 40.9 MB/s eta 0:00:01
     |████████████████████████████████| 148 kB 33.7 MB/s eta 0:00:01
     |████████████████████████████████| 100 kB 17.0 MB/s ta 0:00:01
     |████████████████████████████████| 98 kB 11.0 MB/s eta 0:00:01
  Created wheel for optuna: filename=optuna-2.0.0-py3-none-any.whl size=312964 sha256=fce985ba024e84d10d2d101dd0cbc0fe53a4b0d80b92de75f6237614b1532bb3
  Stored in directory: /root

  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-py3-none-any.whl size=3161 sha256=94a624717b7184f59c6f36a86cc8cc866af6c1bb2c1564a1e0c8fadde516b287
  Stored in directory: /root/.cache/pip/wheels/6a/f5/9c/f8331a854f7a8739cf0e74c13854e4dd7b1af11b04fe1dde13
  Created wheel for pyperclip: filename=pyperclip-1.8.0-py3-none-any.whl size=8692 sha256=669d8e32c333596e1aa3b5cdf07113e7f3cc3f644afa6bd155b1f26625dc8b4d
  Stored in directory: /root/.cache/pip/wheels/26/30/fe/92e2d4b1301ba74c07ea09c9e4c08f5bf12bae9c30319d74c5
  Created wheel for contextvars: filename=contextvars-2.4-py3-none-any.whl size=7665 sha256=892ab52071992fa6ff1a1bbfd7468ea5f129d201ac2281d58e0cf5178e2e9542
  Stored in directory: /root/.cache/pip/wheels/41/11/53/911724983aa48deb94792432e14e518447212dd6c5477d49d3
Successfully built optuna alembic gpustat PrettyTable nvidia-ml-py3 psutil idna-ssl pyperclip contextvars
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.3.0
    Uninstalling tens

In [4]:
!sudo pip3 install optkeras

In [18]:
!sudo pip3 install pandas

In [5]:
#!/usr/bin/env python3
import os
import logging
from pathlib import Path
import requests 
from glob import glob
from zipfile import ZipFile

logging.basicConfig(level=logging.DEBUG)

#Import Pegasus API
from Pegasus.api import *


#Replica Catalog
rc = ReplicaCatalog()
input_files = glob('*.jpg')
input_files.sort()
in_files=[]

for file in input_files:
    in_files.append(File(file))
    rc.add_replica("local", File(file), str(Path(".").resolve() / file))
    
rc.write()


#Transformation
pre_process_resize = Transformation( "preprocess1.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/preprocess1.py",
        is_stageable=True)

pre_process_augment = Transformation( "Augmentation.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/Augmentation.py",
        is_stageable=True)

data_split  = Transformation( "Data_Split.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/Data_Split.py",
        is_stageable=True)

# vgg_model  = Transformation( "VGG_model.py",
#         site = "local",
#         pfn = "/home/scitech/shared-data/CatsAndDogs/VGG_model.py",
#         is_stageable=True)

hpo =  Transformation( "Optuna.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/Optuna.py",
        is_stageable=True)
                    
tc = TransformationCatalog()\
    .add_transformations(pre_process_resize,pre_process_augment,data_split,hpo)\
    .write()

#Workflow
wf = Workflow("Cats_and_Dogs", infer_dependencies=True)


resized_images = File('resized_images.txt')
all_files = [File("resized_{}".format(f.lfn)) for f in in_files]
labels = File('labels.txt')

job_preprocess1 = Job(pre_process_resize)\
                    .add_inputs(*in_files)\
                    .add_outputs(*all_files,resized_images,labels) 

aug_images_txt = File('augmentation.txt')
aug_labels_txt = File('aug_labels.txt')
augmented_files = []
for f in all_files:
    augmented_files.extend([File(str(f).replace("{}".format(os.path.splitext(str(f))[0]), "Aug_{}_{}".format(os.path.splitext(str(f))[0],i))) for i in range(3)])

    
job_preprocess2 = Job(pre_process_augment)\
                    .add_inputs(*all_files,labels)\
                    .add_outputs(aug_images_txt,aug_labels_txt,*augmented_files)

training_data = File('training.pkl')
testing_data = File('testing.pkl')
val_data = File('validation.pkl')

job_data_split = Job(data_split)\
                    .add_inputs(*augmented_files,labels)\
                    .add_outputs(training_data,testing_data,val_data)

model = File('model.h5')

# job_vgg_model = Job(vgg_model)\
#                     .add_inputs(*augmented_files,training_data,testing_data,val_data)\
#                     .add_outputs(model)

optuna_csv = File("Optuna_log_file.csv")

job_hpo = Job(hpo)\
                    .add_args(optuna_csv)\
                    .add_inputs(*augmented_files,training_data,testing_data,val_data)\
                    .add_outputs(model,optuna_csv)


wf.add_jobs(job_preprocess1,job_preprocess2,job_data_split,job_hpo)                                    

In [ ]:
try:
     wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)


################
# pegasus-plan #
################
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.08.24 17:47:45.893 UTC:
2020.08.24 17:47:45.898 UTC:   -----------------------------------------------------------------------
2020.08.24 17:47:45.903 UTC:   File for submitting this DAG to HTCondor           : Cats_and_Dogs-0.dag.condor.sub
2020.08.24 17:47:45.909 UTC:   Log of DAGMan debugging messages                 : Cats_and_Dogs-0.dag.dagman.out
2020.08.24 17:47:45.914 UTC:   Log of 

[######################################------------]  75.0% ..Running (Completed: 18, Queued: 0, Running: 1, Failed: 0)

In [ ]:

         


aug_images_txt = File('augmentation.txt')
aug_labels_txt = File('aug_labels.txt')

job_preprocess2 = Job(pre_process_augment)\
                    .add_inputs(labels,resized_images)\
                    .add_outputs(aug_images_txt,aug_labels_txt)


In [ ]:
augmented_files = []
for f in all_files:
    augmented_files.extend([File(str(f).replace("{}".format(os.path.splitext(str(f))[0]), "Aug_{}_{}".format(os.path.splitext(str(f))[0],i))) for i in range(3)])

In [ ]:
len(augmented_files)

In [ ]:
len(all_files)

In [ ]:

pre_process_augment = Transformation( "Augmentation.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/Augmentation.py",
        is_stageable=True)   


aug_images_txt = File('augmentation.txt')
aug_labels_txt = File('aug_labels.txt')
augmented_files = []
for f in all_files:
    augmented_files.extend([File(str(f).replace("{}".format(os.path.splitext(str(f))[0]), "Aug_{}_{}".format(os.path.splitext(str(f))[0],i))) for i in range(3)])

job_preprocess2 = Job(pre_process_augment)\
                    .add_inputs(*all_files,labels,resized_images)\
                    .add_outputs(aug_images_txt,aug_labels_txt,*augmented_files)
